In [ ]:
%%configure -f
{
"conf": {
    "spark.sql.autoBroadcastJoinThreshold": -1,
    "spark.dynamicAllocation.disableIfMinMaxNotSpecified.enabled": true,
    "spark.dynamicAllocation.enabled": true,
    "spark.dynamicAllocation.minExecutors": 2,
    "spark.dynamicAllocation.maxExecutors": 8
   }
}

In [ ]:
import pyspark.sql.functions as F
from pyspark.ml.functions import vector_to_array
from pyspark.ml.feature import IndexToString, StringIndexerModel
from pyspark.sql.types import FloatType
import joblib
import numpy as np
import shap
from io import BytesIO
import pandas as pd
import h2o

In [ ]:
batch_id = ''
best_iforest_path = ''
prepped_data_path = ''
results_path = ''
features_path = ''
interpret_path = ''
id_feat = ''
id_feat_types = ''
score_threshold = ''
time_slice_folder = ''
number_of_interpret_features = ''
extension_level = ''

In [ ]:
# Initiate logging
import logging
from opencensus.ext.azure.log_exporter import AzureLogHandler
from opencensus.ext.azure.trace_exporter import AzureExporter
from opencensus.trace import config_integration
from opencensus.trace.samplers import AlwaysOnSampler
from opencensus.trace.tracer import Tracer

instrumentation_connection_string = mssparkutils.credentials.getSecretWithLS("keyvault", "AppInsightsConnectionString")
config_integration.trace_integrations(['logging'])

logger = logging.getLogger(__name__)
logger.addHandler(AzureLogHandler(connection_string=instrumentation_connection_string))
logger.setLevel(logging.INFO)

tracer = Tracer(
    exporter=AzureExporter(
        connection_string=instrumentation_connection_string
    ),
    sampler=AlwaysOnSampler()
)

# Spool parameters
run_time_parameters = {'custom_dimensions': {
    'batch_id': batch_id,
    'best_iforest_path': best_iforest_path,
    'prepped_data_path': prepped_data_path,
    'results_path': results_path,
    'features_path': features_path,
    'interpret_path': interpret_path,
    'id_feat': id_feat,
    'id_feat_types': id_feat_types,
    'score_threshold': score_threshold,
    'time_slice_folder': time_slice_folder,
    'notebook_name': mssparkutils.runtime.context['notebookname']
} }
  
logger.info(f"{mssparkutils.runtime.context['notebookname']}: INITIALISED", extra=run_time_parameters)

In [ ]:
if best_iforest_path != "":
    best_iforest_path = "/".join(best_iforest_path.split("/")[:-1]) + "/" + time_slice_folder + "/" + best_iforest_path.split("/")[-1]
    logger.info(f'best_iforest_path = {best_iforest_path}')
if prepped_data_path != "":
    prepped_data_path = "/".join(prepped_data_path.split("/")[:-1]) + "/" + time_slice_folder + "/" + prepped_data_path.split("/")[-1]
    logger.info(f'prepped_data_path = {prepped_data_path}')
if results_path != "":
    results_path = results_path + "/" + time_slice_folder
    logger.info(f'results_path = {results_path}')
if features_path != "":
    features_path = "/".join(features_path.split("/")[:-1]) + "/" + time_slice_folder + "/" + features_path.split("/")[-1]
    logger.info(f'features_path = {features_path}')
if interpret_path != "":
    interpret_path = interpret_path + "/" + time_slice_folder
    logger.info(f'interpret_path = {interpret_path}')

In [ ]:
# Cast parameters
id_feat = [i for i in id_feat.split(",")]
id_feat_types = [i for i in id_feat_types.split(",")]
score_threshold = float(score_threshold)
number_of_interpret_features = int(number_of_interpret_features)
extension_level = int(extension_level)

In [ ]:
df_results = spark.read.parquet(results_path).where(F.col('score')<=score_threshold)

In [ ]:
df_assembled = df_results.join(spark.read.parquet(prepped_data_path), on=id_feat)

In [ ]:
num_feats = len(df_assembled.head(1)[0]['scaled'])
df_unassembled = df_assembled.withColumn('f', vector_to_array("scaled")).select(id_feat + ['score'] + [F.col("f")[i] for i in range(num_feats)])

In [ ]:
if extension_level < 0:
    model_bytes = spark.read.parquet(best_iforest_path).head(1)[0]['model']
    clf = joblib.load(BytesIO(model_bytes))
else:
    clf = None

In [ ]:
feats = np.array([row['feat'] for row in spark.read.parquet(features_path).orderBy('id').collect()])

In [ ]:
top_n_feat_lst = ['feat'+str(i+1) for i in range(number_of_interpret_features)]
top_n_score_lst = ['score'+str(i+1) for i in range(number_of_interpret_features)]
feat_score_lst = list(np.array(list(zip(top_n_feat_lst, top_n_score_lst))).reshape(len(top_n_feat_lst)*2))
dcc_str = ", ".join([x[0]+" "+x[1] for x in zip(id_feat, id_feat_types)]) + ", score float, "
dcc_str += ", ".join([x[0]+" "+x[1] for x in zip(feat_score_lst + ['sum_other_features'], ['string', 'float']*len(top_n_feat_lst) + ['float'])])

In [ ]:
def ijungle_interpret(id_feat, clf, feats, top_n_feat_lst, top_n_score_lst, feat_score_lst, ex_level):
    def _fun(iterator):
        if ex_level < 0:
            explainer = shap.TreeExplainer(clf)
        for pdf in iterator:
            pdf.set_index(id_feat + ['score'], inplace=True)
            if ex_level < 0:
                shap_values = explainer.shap_values(pdf)
            else:
                #Dummy variables so PowerBI doesn't break
                shap_values = np.array([[1./len(feats) for _ in feats] for _ in range(pdf.shape[0])])
            top_feats = feats[shap_values.argsort()[:,:len(top_n_feat_lst)]]
            pdf_out = pdf.reset_index()
            pdf_out = pdf_out[id_feat + ['score']]
            norm_fracs = np.array(pdf_out['score'])/np.sum(shap_values,axis=1)
            top_scores = np.sort(shap_values)[:,:len(top_n_feat_lst)]*norm_fracs.reshape(norm_fracs.shape[0],-1)
            pdf_out[top_n_feat_lst] = top_feats
            pdf_out[top_n_score_lst] = top_scores
            pdf_out['sum_other_features'] = np.sum(np.sort(shap_values)[:,len(top_n_feat_lst):],axis=1)*norm_fracs
            pdf_out = pdf_out[id_feat + ['score'] + feat_score_lst + ['sum_other_features']]
            yield(pdf_out)
    return(_fun)

In [ ]:
df_interpret = df_unassembled.mapInPandas(ijungle_interpret(id_feat, clf, feats, top_n_feat_lst, top_n_score_lst, feat_score_lst, extension_level), dcc_str)
model = StringIndexerModel.load("/".join(best_iforest_path.split("/")[:-2]) + '/' + '_feature_engineering_indexer_issuer_id.pkl')
inverter = IndexToString(inputCol="issuer_id_indexed", outputCol="issuer_id", labels=model.labels)
df_interpret = inverter.transform(df_interpret)

In [ ]:
with tracer.span('Saving interpret data to ADLS'):
    df_interpret.write.mode('overwrite').parquet(interpret_path)

In [ ]:
new_rows = ['issuer_id','issued_date']
new_rows += [F.array(F.array(F.lit('total_score'),'score'),*[F.array(feat, score) for idx, (feat, score) in enumerate(zip(top_n_feat_lst, top_n_score_lst))],F.array(F.lit('sum_other_features'),'sum_other_features')).alias("merged_rows")]
df_interpret_by_row = df_interpret.select(*new_rows)
df_interpret_by_row = df_interpret_by_row.select('issuer_id','issued_date',F.explode('merged_rows').alias('key_value'))
df_interpret_by_row = df_interpret_by_row.withColumn('feature',df_interpret_by_row['key_value'].getItem(0)).withColumn('score',df_interpret_by_row['key_value'].getItem(1).cast(FloatType())).drop('key_value')

In [ ]:
interpret_path_by_row = '/'.join(interpret_path.split("/")[:-1]) + "_exploded" + "/" + interpret_path.split("/")[-1]
logger.info(interpret_path)
logger.info(interpret_path_by_row)

In [ ]:
with tracer.span('Saving interpret by row data to ADLS'):
    df_interpret_by_row.write.mode('overwrite').parquet(interpret_path_by_row)

In [ ]:
# serverless SQL config
import pyodbc
database = 'eiad'
driver= '{ODBC Driver 17 for SQL Server}'

sql_user_name = mssparkutils.credentials.getSecretWithLS("keyvault", "SynapseSQLUserName")
sql_user_pwd = mssparkutils.credentials.getSecretWithLS("keyvault", "SynapseSQLPassword")
serverless_sql_endpoint = mssparkutils.credentials.getSecretWithLS("keyvault", "SyanpseServerlessSQLEndpoint")

In [ ]:
def generate_schema_string(dataframe):
    schema_string = ""
    for name in dataframe.schema.fieldNames():
        schema_string += "[" + name + "] "
        datatype = str(dataframe.schema[name].dataType.simpleString())
        if datatype == 'double': datatype = 'float'
        if datatype == 'string': datatype = 'nvarchar(MAX)'
        if datatype == 'timestamp': datatype = 'datetime2(7)'
        schema_string += datatype + ", "
    return schema_string[:-2]

In [ ]:
with tracer.span('Creating SQL table for interpret data'):
    table_name = interpret_path.split('/')[3] + '_' + interpret_path.split('/')[2].split('@')[0] + '_' + interpret_path.split('/')[4] + '_' + interpret_path.split('/')[5]
    schema_string = generate_schema_string(df_interpret)
    drop_table_command = f"DROP EXTERNAL TABLE [{table_name}]"
    location = "/".join([i for idx, i in enumerate(interpret_path.split('/')) if idx > 2])
    df_sql_command = f"CREATE EXTERNAL TABLE [{table_name}] ({schema_string}) WITH (LOCATION = '{location}/**', DATA_SOURCE = [output_<<STORAGE_ACCOUNT_NAME>>_dfs_core_windows_net], FILE_FORMAT = [SynapseParquetFormat])"
    with pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+serverless_sql_endpoint+';PORT=1433;DATABASE='+database+';UID='+sql_user_name+';PWD='+ sql_user_pwd) as conn:
        with conn.cursor() as cursor:
            try:
                cursor.execute(drop_table_command)
            except:
                pass
            cursor.execute(df_sql_command)

In [ ]:
with tracer.span('Creating SQL table for interpret by row data'):
    table_name = interpret_path_by_row.split('/')[3] + '_' + interpret_path_by_row.split('/')[2].split('@')[0] + '_' + interpret_path_by_row.split('/')[4] + '_' + interpret_path_by_row.split('/')[5]
    schema_string = generate_schema_string(df_interpret_by_row)
    drop_table_command = f"DROP EXTERNAL TABLE [{table_name}]"
    location = "/".join([i for idx, i in enumerate(interpret_path_by_row.split('/')) if idx > 2])
    df_sql_command = f"CREATE EXTERNAL TABLE [{table_name}] ({schema_string}) WITH (LOCATION = '{location}/**', DATA_SOURCE = [output_<<STORAGE_ACCOUNT_NAME>>_dfs_core_windows_net], FILE_FORMAT = [SynapseParquetFormat])"
    with pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+serverless_sql_endpoint+';PORT=1433;DATABASE='+database+';UID='+sql_user_name+';PWD='+ sql_user_pwd) as conn:
        with conn.cursor() as cursor:
            try:
                cursor.execute(drop_table_command)
            except:
                pass
            cursor.execute(df_sql_command)